# Task 3. Departement prediction

In [ ]:
from __future__ import absolute_import, division, print_function

import argparse
import csv
import json
import logging
import os
import random
import sys

import numpy as np
import torch
import torch.nn.functional as F

# mult processing
# import torch.multiprocessing as mp

# for horovod distributed training
# import torch.utils.data.distributed

from transformers import (WEIGHTS_NAME, AdamW, BertConfig,
                          BertForTokenClassification, BertTokenizer,
                          get_linear_schedule_with_warmup, 
                          BertPreTrainedModel, BertModel)

from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
# from transformers.models.bert.modeling_bert import Sentence_classifier


from torch import nn
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
# from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

from seqeval.metrics import classification_report
from seqeval.metrics import sequence_labeling

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)
import codecs

try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle
    
import glob
import time

import easydict

import six
import tqdm
from tqdm import tqdm, trange
import collections

from seqeval.metrics import classification_report

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)


import re

import shutil

In [ ]:
class TrainingInstance(object):
    """A single training instance (sentence pair)."""
    def __init__(self, input_ids, input_mask, segment_ids, labels):
        self.input_ids=input_ids
        self.input_mask=input_mask
        self.segment_ids=segment_ids
        self.labels=labels

## cls classifier

In [ ]:
class CLS_classifier(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.classifier = nn.Linear(config.hidden_size, 8) # 8개 과 구분 task
        self.init_weights()
        
    def forward(self, input_ids, label_ids, 
                attention_mask=None, token_type_ids=None,  # 220315 segment_id
                position_ids=None, head_mask=None, inputs_embeds=None,
                output_attentions=None, output_hidden_states=None,
                return_dict=None):
        
        outputs = self.bert(
            input_ids = input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids,
            position_ids = position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        pooled_output = outputs[1]
      
        seq_relationship_scores = self.classifier(pooled_output)
        batsize = seq_relationship_scores.size()[0]
        
        # labels
        labels = torch.zeros(batsize, dtype=torch.long, device='cuda')
        for l in range(len(labels)):
            labels[l]=label_ids[l][0]
            
        loss_fct = CrossEntropyLoss()
        
        loss = loss_fct(seq_relationship_scores.view(-1, 8), labels.view(-1))

        probs = seq_relationship_scores.detach().cpu()
        probs = nn.functional.log_softmax(probs, dim=-1) 

        return loss, probs

## Train

In [ ]:
def train(args):
    ############################################################################
    ##              Multi-GPUs, Distributed settings
    ############################################################################ 
    if args.server_ip and args.server_port:
    # Distant debugging - see https://code.visualstudio.com/docs/python/debugging#_attach-to-a-local-script
        import ptvsd
        print("Waiting for debugger attach")
        ptvsd.enable_attach(address=(args.server_ip, args.server_port), redirect_output=True)
        ptvsd.wait_for_attach()
    
    if args.local_rank == -1 or args.no_cuda:
        device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        n_gpu = torch.cuda.device_count()
    else:
        torch.cuda.set_device(args.local_rank)
        device = torch.device("cuda", args.local_rank)
        n_gpu = 1
        # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
        torch.distributed.init_process_group(backend='nccl')
    logger.info("device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
        device, n_gpu, bool(args.local_rank != -1), args.fp16))
    
    print("n_gpu: ", n_gpu)
    
    
    ############################################################################
    ##              batch size, gradient_accumulation_steps
    ############################################################################ 
    if args.gradient_accumulation_steps < 1:
        raise ValueError("Invalid gradient_accumulation_steps parameter: {}, should be >= 1".format(
                            args.gradient_accumulation_steps))
    args.train_batch_size = args.train_batch_size // args.gradient_accumulation_steps
    print("args.train_batch_size: ", args.train_batch_size)
    
    ###############################################################
    #                Functions for Dataload (load cache data)
    ###############################################################
    # 파일 목록 획득
    def get_file_arrays(data_path):
        data_path = data_path.split(",")
        print("data_path: ", data_path)
        
        cache_files_all = []
        for i in range(len(data_path)):
            path = data_path[i].strip()
            files = glob.glob(path)
            cache_files_all = cache_files_all + files
        cache_files_all.sort()
        
        print("Got " + str(len(cache_files_all)) + " cache_files")
        
        return cache_files_all
    
    # split file arrays
    def split_file_array(cache_files, num_files_on_memory):
        cache_file_groups = []
        for i in range(0, len(cache_files), num_files_on_memory):
            start = i
            if (i+num_files_on_memory) < len(cache_files):
                end = i+num_files_on_memory
            else:
                end = len(cache_files)
            cache_file_groups.append(cache_files[start:end])                
        return cache_file_groups
    
    
    def read_files(cache_files_pieces):
#         print("READING NEXT FILES OF GROUP...")
        train_features = []
        for c in range(len(cache_files_pieces)):
            with open(cache_files_pieces[c], 'rb') as input:
                train_features = train_features + pickle.load(input)
            print("cache_files_pieces[c]: ", cache_files_pieces[c])
        print("len(train_features): ", len(train_features))        
        
        num_train_examples = len(train_features)
        
        all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
        all_labels = torch.tensor([f.labels for f in train_features], dtype=torch.long)
        
        
        all_input_ids = torch.squeeze(all_input_ids)
        all_input_mask = torch.squeeze(all_input_mask)
        all_segment_ids = torch.squeeze(all_segment_ids)
        all_labels = torch.squeeze(all_labels)
        
        train_data = TensorDataset(
            all_input_ids, all_input_mask, all_segment_ids, all_labels
        )
        
        if args.local_rank == -1:
            train_sampler = RandomSampler(train_data)
        else:
            train_sampler = DistributedSampler(train_data)
        
        print("len(train_data): ", len(train_data))
        print("args.train_batch_size: ", args.train_batch_size)
        
        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=args.train_batch_size)
        
        print("len(train_dataloader): ", len(train_dataloader))
        
        return train_dataloader, num_train_examples
    
    ############################################################################
    ##                    Load Training data
    ############################################################################
    label_list = ["0", "1"] # not extraction, extraction
    num_labels = len(label_list) # + 1 NER 에서는 [PAD] 때문에 +1 했었음
    
    
    ###############################################################
    #            Dataload (load cache data) - Train data
    ###############################################################
    # 학습 파일 목록 획득
    cache_files = get_file_arrays(args.data_dir)
    print("cache_files: ", cache_files)
    train_dataloader, num_train_examples = read_files(cache_files)
    

    ############################################################################
    ##                      Prepare model
    ############################################################################
    print("loading weights from checkpoint (", args.checkpoint, ")")
    config = BertConfig.from_pretrained(args.checkpoint, num_labels=num_labels)
    model = CLS_classifier.from_pretrained(args.checkpoint,
              from_tf = False,
              config = config)
    print("loaded weights from checkpoint (", args.checkpoint, ")")
    
    if args.local_rank == 0:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab
    print("device: ", device)
    
    model.to(device)
    
    
    ############################################################################
    ##                        Optimizer
    ############################################################################
    num_train_optimization_steps = int(
        num_train_examples / args.train_batch_size / args.gradient_accumulation_steps) * args.num_train_epochs
    if args.local_rank != -1:
        num_train_optimization_steps = num_train_optimization_steps // torch.distributed.get_world_size()

    if args.local_rank not in [-1, 0]:
        torch.distributed.barrier()  # Make sure only the first process in distributed training will download model & vocab
    
    
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias','LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    warmup_steps = int(args.warmup_proportion * num_train_optimization_steps)
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=num_train_optimization_steps)
    
    
    ############################################################################
    ##              save model
    ############################################################################
    def save_model(args, global_step):
        if not os.path.exists(args.output_dir+"/"+str(global_step)):
            os.makedirs(args.output_dir+"/"+str(global_step))

        # save model
        # Save a trained model and the associated configuration
        model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
        model_to_save.save_pretrained(args.output_dir+"/"+str(global_step))

    
    ############################################################################
    ##              Start training
    ############################################################################
    global_step = -1
    nb_tr_steps = 0
    tr_loss = 0
    
    # save loss
    file = open(args.output_dir+"/loss.txt", "w")
    file.write("globalstep\tepoch\tstep\tloss\n")
    file.close()
    
    
    args.save_file_limit
    min_train_loss = 1000000
    save_checkpoints = []
    save_loss = []
    
    
    model.train()
    for epoch in trange(int(args.num_train_epochs), desc="Epoch"):
        tr_loss = 0
        nb_tr_examples, nb_tr_steps = 0, 0
        for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
            batch = tuple(t.to(device) for t in batch)
            input_ids, input_mask, segment_ids, label = batch
            
            loss, probs = model(input_ids=input_ids, label_ids=label, 
                attention_mask=input_mask, token_type_ids=segment_ids,  # 220315 segment_id
                position_ids=None, head_mask=None, inputs_embeds=None,
                output_attentions=None, output_hidden_states=None,
                return_dict=None)
            
            #print("probs: ", probs)
        
            
            if n_gpu > 1:
                loss = loss.mean() # mean() to average on multi-gpu.
                
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
            else:
                current_loss = loss.item()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

            tr_loss += loss.item()
            nb_tr_examples += input_ids.size(0)
            nb_tr_steps += 1
            if (step + 1) % args.gradient_accumulation_steps == 0:
                # save loss
                file = open(args.output_dir+"/loss.txt", "a")
                file.write(str(global_step+1)+"\t"+str(epoch)+"\t"+str(step)+"\t"+str(loss.item())+"\n")
                file.close()
                
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1
                      
    save_model(args, global_step)

In [ ]:
def main(args):
    parser = argparse.ArgumentParser()
    
    print("args.output_dir: ", args.output_dir)
    
    # check directories
    if not args.do_train and not args.do_eval:
        raise ValueError("At least one of `do_train` or `do_eval` must be True.")
    if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train:
        raise ValueError("Output directory ({}) already exists and is not empty.".format(args.output_dir))
    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
            
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    
    if args.do_train:
        train(args)
    
    if args.do_eval:
        evaluate(args)

# finetune

In [ ]:
data_dirs = [
    './cache/bertbase_cased/train/*.cache',
    './cache/mbert_cased/train/*.cache',
    './cache/biobert/train/*.cache',
    './cache/kobert/train/*.cache',
]

checkpoints = [
    '../pretrainBERT/ver9.1.4/521121_epoch2',  # bert cased (pretrained on SNUH visit records)
    '../pretrainBERT/ver8.1.4/1142642_epoch2',  # mbert cased (pretrained on SNUH visit records)
    '../pretrainBERT/ver11.1.4/521079_epoch2',  # biobert (pretrained on SNUH visit records)
    '../pretrainBERT/ver12.1.4/407013_epoch2',  # kobert (pretrained on SNUH visit records)
]

assert len(data_dirs)==len(checkpoints)


for d in range(len(data_dirs)):
    output_dir = "./finetuned/"+"_".join(checkpoints[d].split("/")[-2:])
    print("output_dir: ", output_dir)

    args = easydict.EasyDict({
        'data_dir':data_dirs[d],
        'train_batch_size':5,             # train_batch_size/gradient_accumulation_steps
        'gradient_accumulation_steps':1,  # become update step
        'save_globalstep':1,
        'checkpoint':checkpoints[d],
        'output_dir':output_dir,
        'save_step':1,
        'num_files_on_memory':2,
        
        'do_train':True,
        'do_eval':False,
        'do_lower_case':True,
        'fp16_allreduce':False,
        'seed':42,
        'no_cuda':False,
        'use_adasum':False,
        'learning_rate':3e-05,
        'warmup_proportion':1.00,
        'num_train_epochs':2,
        'save_file_limit':1,
        
        'weight_decay':0.01,
        'adam_epsilon':1e-8,
        'gradient_predivide_factor':1.0,
        'fp16':False,
        'max_grad_norm':1.0,
        'num_files_on_memory':2,
        'local_rank':-1,
        'fp16_opt_level':'O1',
        'loss_scale':0,
        
        'server_ip':None,
        'server_port':None,
        'cuda':True,
    })
    
    
    # configs
    if "mbert_cased" == str(data_dirs[d].split("/")[2]):
        print("multilingualbert configs")
        # multilingual bert config
        args["attention_probs_dropout_prob"] = 0.1
        args["directionality"] = "bidi"
        args["hidden_act"] = "gelu"
        args["hidden_dropout_prob"] = 0.1
        args["hidden_size"] = 768
        args["initializer_range"] = 0.02
        args["intermediate_size"] = 3072
        args["max_position_embeddings"] = 512
        args["num_attention_heads"] =  12
        args["num_hidden_layers"] = 12
        args["pooler_fc_size"] = 768
        args["pooler_num_attention_heads"] = 12
        args["pooler_num_fc_layers"] = 3
        args["pooler_size_per_head"] = 128
        args["pooler_type"] = "first_token_transform"
        args["type_vocab_size"] = 2
        args["vocab_size"] = 119547
        args["cls_id"] = 101
        args["sep_id"] = 102
        
    elif "mbert_uncased" == str(data_dirs[d].split("/")[2]):
        print("multilingualbert configs")
        # multilingual bert config
        args["attention_probs_dropout_prob"] = 0.1
        args["directionality"] = "bidi"
        args["hidden_act"] = "gelu"
        args["hidden_dropout_prob"] = 0.1
        args["hidden_size"] = 768
        args["initializer_range"] = 0.02
        args["intermediate_size"] = 3072
        args["layer_norm_eps"] = 1e-12
        args["max_position_embeddings"] = 512
        args["model_type"] = "bert"
        args["num_attention_heads"] =  12
        args["num_hidden_layers"] = 12
        args["pad_token_id"] = 0
        args["pooler_fc_size"] = 768
        args["pooler_num_attention_heads"] = 12
        args["pooler_num_fc_layers"] = 3
        args["pooler_size_per_head"] = 128
        args["pooler_type"] = "first_token_transform"
        args["type_vocab_size"] = 2
        args["vocab_size"] = 105879
        args["cls_id"] = 101
        args["sep_id"] = 102
        
    elif "bertbase_uncased" == str(data_dirs[d].split("/")[2]):
        args['vocab_size']=30522 # bert-base-uncased
        args['hidden_size']=768
        args['num_hidden_layers']=12
        args['num_attention_heads']=12
        args['hidden_act']='gelu'
        args['intermediate_size']=3072
        args['hidden_dropout_prob']=0.1
        args['attention_probs_dropout_prob']=0.1
        args['max_position_embeddings']=512
        args['type_vocab_size']=2
        args['initializer_range']=0.02
        args['layer_norm_eps']=1e-12
        args['gradient_checkpointing']=None
        args['position_embedding_type']=None
        args['use_cache']=None
        args['classifier_dropout']=None
        args["cls_id"] = 101
        args["sep_id"] = 102

    elif "bertbase_cased" == str(data_dirs[d].split("/")[2]):
        args['vocab_size']=28996 # bert-base-cased
        args['attention_probs_dropout_prob']=0.1
        args['hidden_act']='gelu'
        args['hidden_dropout_prob']=0.1
        args['hidden_size']=768
        args['initializer_range']=0.02
        args['intermediate_size']=3072
        args['layer_norm_eps']=1e-12
        args['max_position_embeddings']=512
        args['model_type']="bert"
        args['num_hidden_layers']=12
        args['num_attention_heads']=12
        args['pad_token_id']=0        
        args['type_vocab_size']=2
        args['gradient_checkpointing']=None
        args['position_embedding_type']=None
        args['use_cache']=None
        args['classifier_dropout']=None
        args["cls_id"] = 101
        args["sep_id"] = 102
        
    # configs
    elif "biobert" == str(data_dirs[d].split("/")[2]):
        # BioBERT
        print("biobert configs")
        args["attention_probs_dropout_prob"]= 0.1
        args["hidden_act"] = "gelu"
        args["hidden_dropout_prob"] = 0.1
        args["hidden_size"] = 768
        args["initializer_range"] = 0.02
        args["intermediate_size"] = 3072
        args["max_position_embeddings"] = 512
        args["num_attention_heads"] = 12
        args["num_hidden_layers"] = 12
        args["type_vocab_size"] = 2
        args["vocab_size"] = 28996
        args["cls_id"] = 101
        args["sep_id"] = 102
        
    elif "kobert" == str(data_dirs[d].split("/")[2]):
        print("kobert configs")
        args["attention_probs_dropout_prob"]= 0.1
        args["gradient_checkpointing"]= False
        args["hidden_act"]= "gelu"
        args["hidden_dropout_prob"]= 0.1
        args["hidden_size"]= 768
        args["initializer_range"]= 0.02
        args["intermediate_size"]= 3072
        args["layer_norm_eps"]= 1e-12
        args["max_position_embeddings"]= 512
        args["model_type"]= "bert"
        args["num_attention_heads"]= 12
        args["num_hidden_layers"]= 12
        args["pad_token_id"]= 1
        args["type_vocab_size"]= 2
        args["vocab_size"]= 8002
        args["author"]= "Heewon Jeon(madjakarta@gmail.com)"
        args["kobert_version"]= 1.0
        args["cls_id"] = 2
        args["sep_id"] = 3
                
    else:
        print("MY BERT")
        args['vocab_size']=30014
        args['hidden_size']=768
        args['num_hidden_layers']=12
        args['num_attention_heads']=12
        args['hidden_act']='gelu'
        args['intermediate_size']=3072
        args['hidden_dropout_prob']=0.1
        args['attention_probs_dropout_prob']=0.1
        args['max_position_embeddings']=512
        args['type_vocab_size']=2
        args['initializer_range']=0.02
        args['layer_norm_eps']=1e-12
        args['gradient_checkpointing']=None
        args['position_embedding_type']=None
        args['use_cache']=None
        args['classifier_dropout']=None
        args["cls_id"] = 4
        args["sep_id"] = 5
        
    main(args)